In [224]:
import pandas as pd

df_operations = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Operations')

In [225]:
df_operations = df_operations[['event', 'Category']]
mask = df_operations['Category'].notna()
df_operations_u = df_operations[mask].reset_index(drop=True)
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [226]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'ODBC Driver 17 for SQL Server']


In [227]:
driver = 'ODBC Driver 17 for SQL Server'
server = 'BKZTKDSDB41,21433'
database = 'PIFD'

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# SQL-запрос
query = """
SELECT 
    AFE.name COLLATE DATABASE_DEFAULT AS [equipment],
    AFET.name COLLATE DATABASE_DEFAULT AS [event],
    DATEADD(SECOND, (AVEvF.starttime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01') AS starttime,
    IIF(AVEvF.endtime > 2638317188000000000, NULL, 
        DATEADD(SECOND, (AVEvF.endtime / 10000000) - 63240134400 + 6 * 3600, '2005-01-01')) AS endtime
FROM PIFD.dbo.AFEventFrame AS AVEvF
LEFT JOIN PIFD.dbo.AFElementTemplate AS AFET 
    ON AVEvF.fktemplateid = AFET.rid
LEFT JOIN PIFD.dbo.AFElement AS AFE 
    ON AVEvF.fkprimaryreferencedelement = AFE.rid
WHERE AFE.name COLLATE DATABASE_DEFAULT IN (
    SELECT DISTINCT equipment COLLATE DATABASE_DEFAULT 
    FROM PI_EXTRA.dbo.DR_REL_EQUIPMENT_EVENT
)
AND AFE.id IN (
    SELECT fkelementid 
    FROM PIFD.dbo.AFElementVersion 
    WHERE fktemplateid = (
        SELECT id 
        FROM PIFD.dbo.AFElementTemplate 
        WHERE name = 'RIG' 
        AND fkdatabaseid = (
            SELECT id FROM PIFD.dbo.AFDatabase WHERE name = 'DEM_DR'
        )
    )
)
"""

# Загрузка в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]
        print(df_osi_operations.head())  # Проверка результата
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\3572333381.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_osi_operations = pd.read_sql(query, conn) # pyright: ignore[reportArgumentType]


       equipment         event           starttime             endtime
0  DBKAZ40 BL №1     RP_REPAIR 2021-08-20 12:00:00 2021-09-02 17:00:00
1  DBKAZ40 BL №1  CV_LONG_STOP 2020-04-27 08:00:00 2020-08-15 08:00:00
2  DBKAZ40 BL №1        WT_PPR 2020-02-18 08:00:00 2020-02-18 09:00:00
3  DBKAZ40 BL №1      WT_LUNCH 2020-02-18 12:00:00 2020-02-18 13:00:00
4  DBKAZ40 BL №1        WT_PPR 2020-02-18 20:00:00 2020-02-18 21:00:00


In [228]:
df_osi_operations['duration'] = df_osi_operations['endtime'] - df_osi_operations['starttime']

df_osi_operations['duration'] = df_osi_operations['duration'].dt.total_seconds() / 3600

df_osi_operations


,equipment,event,starttime,endtime,duration
0,DBKAZ40 BL №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0
1,DBKAZ40 BL №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0
2,DBKAZ40 BL №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0
3,DBKAZ40 BL №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0
4,DBKAZ40 BL №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0
...,...,...,...,...,...
556847,ЯЯЯЯ-ФЕЙК,DR_CORE_DRILLING,2024-09-29 21:00:00,2024-09-29 23:30:00,2.5
556848,ЯЯЯЯ-ФЕЙК,DR_REAMING,2024-09-30 11:30:00,2024-09-30 13:30:00,2.0
556849,ЯЯЯЯ-ФЕЙК,DR_PUMP_REPLACEMENT,2024-09-30 14:00:00,2024-09-30 15:30:00,1.5
556850,ЯЯЯЯ-ФЕЙК,DR_WELL_FLUSHING,2024-10-01 12:30:00,2024-10-01 15:00:00,2.5


In [229]:
df_rigs = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Rigs-Name')
df_rigs

,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,BurGeoProekt,PRAKLA_01-BGP,NaN,Direct
1,BurGeoProekt,PRAKLA_02-BGP,NaN,Direct
2,BurGeoProekt,PRAKLA_03-BGP,NaN,Direct
3,BurGeoProekt,PRAKLA_04-BGP,NaN,Direct
4,BurGeoProekt,ZIF1200_01-BGP,ZIF 1200 №1 Бургеопроект,Direct
...,...,...,...,...
157,VolkovGeology_#5,ZIF1200_36-E05,NaN,Direct
158,VolkovGeology_#5,ZIF1200_37-E05,NaN,Direct
159,VolkovGeology_#5,ZIF1200_38-E05,NaN,Direct
160,VolkovGeology_#5,ZIF1200_42-E05,NaN,Direct


In [230]:
df_circ_data = pd.read_excel(
    "k:/DOP/OED/METHOD&TOOLS/3 - PROJECTS/2 - ON GOING/2 - OE/2502 DIGITAL PROJECTS YEVGENIY/TRS/TRS_relations.xlsx",
    sheet_name='Standart avarage meters')
df_circ_data

,Circ,"Standard avarage drilling, m/h","time to well drill, h"
0,Direct,2.7,210
1,RC,5.4,87


In [231]:
df_osi_operations['endtime'].max()

Timestamp('2025-06-03 09:00:00')

In [232]:
database_a = 'acqkzdem1'
# SQL-запрос
query_2 = """
SELECT 
    HOLEID,
    DrillCompany,
    DrillRig,
    HolePurpose,
    HoleStatus,
    ENDDATE,
    [DEPTH]
FROM [acqkzdem1].[dbo].[AREVA_V_ARCGIS_DH_DEM]
"""

# Строка подключения
conn_str = f'DRIVER={{{driver}}};SERVER={server};DATABASE={database_a};Trusted_Connection=yes;'

# Загрузка данных в DataFrame
try:
    with pyodbc.connect(conn_str) as conn:
        df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]
        print(df_meters.head())  # Проверка
except Exception as e:
    print("Ошибка при подключении или выполнении запроса:", e)

C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\1744419047.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_meters = pd.read_sql(query_2, conn) # pyright: ignore[reportArgumentType]


           HOLEID       DrillCompany         DrillRig HolePurpose  \
0     MCI_N_227_1  TechnoService-Eng   ZIF1200_08-TSE       PIEZO   
1     MCI_N_233_1        Mechkolonna  ZIF1200_19-MK49       PIEZO   
2     MCI_N_236_1       BurGeoProekt   ZIF1200_09-BGP       PIEZO   
3     MCI_N_245_1              KATCO    ZMO1500-5-KAT       PIEZO   
4  MCI01_01_01_B_              KATCO             None      INJECT   

     HoleStatus    ENDDATE  DEPTH  
0        LIQUID 2024-11-16  421.0  
1      ACCEPTED 2025-01-01  424.0  
2      ACCEPTED 2024-12-09  417.0  
3  CONSTRUCTION        NaT  417.0  
4      ACCEPTED 2013-09-16  415.0  


In [233]:
df_meters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17906 entries, 0 to 17905
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   HOLEID        17906 non-null  object        
 1   DrillCompany  17904 non-null  object        
 2   DrillRig      17762 non-null  object        
 3   HolePurpose   17906 non-null  object        
 4   HoleStatus    17906 non-null  object        
 5   ENDDATE       17835 non-null  datetime64[ns]
 6   DEPTH         17813 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 979.4+ KB


df_operations     - Название операций <br>
df_operations_u   - Название операций уникальные значения <br>
df_osi_operations - Время операций <br>
df_rigs           - Название агрегатов для связи <br>
df_circ_data      - Стандартные коэфициенты в зависим от циркуляции (скорость бурения в час, и время на одну скважину - час) <br>
df_meters         - Пробуренные метры

## Идея 
1 связать df_osi_operations - df_rigs (Взять за основу Rig-AcQuire не пустые)
2 связать df_osi_operations - df_df_operations_u


In [234]:
mask = df_rigs['Rig-AcQuire'].notna()
df_rigs = df_rigs[mask].reset_index(drop=True)
df_rigs.info()
df_osi_operations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   DrillCompany         162 non-null    object
 1   Rig-AcQuire          162 non-null    object
 2   Rig-osiDEM           74 non-null     object
 3   Tipe of circulation  162 non-null    object
dtypes: object(4)
memory usage: 5.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556852 entries, 0 to 556851
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   equipment  556852 non-null  object        
 1   event      556852 non-null  object        
 2   starttime  556852 non-null  datetime64[ns]
 3   endtime    556844 non-null  datetime64[ns]
 4   duration   556844 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(2)
memory usage: 21.2+ MB


In [235]:
df_osi_operations['equipment'] = df_osi_operations['equipment'].str.strip().str.lower()
df_rigs['Rig-osiDEM'] = df_rigs['Rig-osiDEM'].str.strip().str.lower()

df_merged = df_osi_operations.merge(
    df_rigs,
    left_on='equipment',
    right_on='Rig-osiDEM',
    how='inner'  # или 'inner', если нужны только совпадающие строки
)

df_merged

,equipment,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Rig-osiDEM,Tipe of circulation
0,dbkaz40 bl №1,RP_REPAIR,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
1,dbkaz40 bl №1,CV_LONG_STOP,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
2,dbkaz40 bl №1,WT_PPR,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
3,dbkaz40 bl №1,WT_LUNCH,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
4,dbkaz40 bl №1,WT_PPR,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,dbkaz40 bl №1,RC
...,...,...,...,...,...,...,...,...,...
505438,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_5_2,2025-06-01 16:00:00,2025-06-01 19:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505439,пбу змо-1500пс №6,WT_WAITING_FOR_LOGGING_RESULT_5_2,2025-06-01 19:00:00,2025-06-01 21:00:00,2.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505440,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-01 22:00:00,2025-06-02 01:00:00,3.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct
505441,пбу змо-1500пс №6,AS_MAST_DOWN,2025-06-02 02:00:00,2025-06-02 09:00:00,7.0,KATCO,ZMO1500-6-KAT,пбу змо-1500пс №6,Direct


In [236]:
df_temp = df_merged.drop(columns=['equipment', 'Rig-osiDEM'])

In [237]:
df_operations_u

,event,Category
0,LG_PHASE_1,Planned_downtime
1,LG_PHASE_2,Planned_downtime
2,LG_PHASE_3,Planned_downtime
3,LG_PHASE_4_1,Planned_downtime
4,LG_PHASE_5_2,Planned_downtime
5,RP_BROKEN_CABLE,Unplanned_downtime_losses
6,RP_BROKEN_ELECTRICAL,Unplanned_downtime_losses
7,RP_BROKEN_GENERATOR,Unplanned_downtime_losses
8,RP_BROKEN_HYDRAULIC,Unplanned_downtime_losses
9,RP_BROKEN_PT_1200,Unplanned_downtime_losses


In [238]:
df_temp['event'] = df_temp['event'].str.strip().str.lower()
df_operations_u['event'] = df_operations_u['event'].str.strip().str.lower()

df_operations_total = df_temp.merge(df_operations_u, on='event', how='left')

df_operations_total['Category'] = df_operations_total['Category'].fillna('Speed_losess')

In [239]:
df_operations_copy = df_operations_total.copy()
df_operations_total

,event,starttime,endtime,duration,DrillCompany,Rig-AcQuire,Tipe of circulation,Category
0,rp_repair,2021-08-20 12:00:00,2021-09-02 17:00:00,317.0,KATCO,DBKAZ40-1,RC,Unplanned_downtime_losses
1,cv_long_stop,2020-04-27 08:00:00,2020-08-15 08:00:00,2640.0,KATCO,DBKAZ40-1,RC,Speed_losess
2,wt_ppr,2020-02-18 08:00:00,2020-02-18 09:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
3,wt_lunch,2020-02-18 12:00:00,2020-02-18 13:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
4,wt_ppr,2020-02-18 20:00:00,2020-02-18 21:00:00,1.0,KATCO,DBKAZ40-1,RC,Planned_downtime
...,...,...,...,...,...,...,...,...
505438,wt_waiting_for_logging_5_2,2025-06-01 16:00:00,2025-06-01 19:00:00,3.0,KATCO,ZMO1500-6-KAT,Direct,Unplanned_downtime_losses
505439,wt_waiting_for_logging_result_5_2,2025-06-01 19:00:00,2025-06-01 21:00:00,2.0,KATCO,ZMO1500-6-KAT,Direct,Unplanned_downtime_losses
505440,as_mast_down,2025-06-01 22:00:00,2025-06-02 01:00:00,3.0,KATCO,ZMO1500-6-KAT,Direct,Speed_losess
505441,as_mast_down,2025-06-02 02:00:00,2025-06-02 09:00:00,7.0,KATCO,ZMO1500-6-KAT,Direct,Speed_losess


In [240]:
# Убедимся, что endtime — datetime (хотя уже подтверждено)
df_operations_total['endtime'] = pd.to_datetime(df_operations_total['endtime'])
df_operations_total = df_operations_total.dropna(subset=['endtime'])

# Добавим год и месяц
df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')


pivot_df = df_operations_total.pivot_table(
    index=['DrillCompany', 'Rig-AcQuire', 'year', 'month', 'Tipe of circulation'],
    columns='Category',
    values='duration',
    aggfunc='sum',
    fill_value=0  # если нужно подставить 0 вместо NaN
).reset_index()

pivot_df = pivot_df.sort_values(
    by=['DrillCompany', 'Rig-AcQuire', 'year', 'month'],
    ascending=[True, True, True, True]
).reset_index(drop=True)


pivot_df


C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\1988683443.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['year'] = df_operations_total['endtime'].dt.year.astype('int16')
C:\Users\ykarabekov\AppData\Local\Temp\ipykernel_20264\1988683443.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_operations_total['month'] = df_operations_total['endtime'].dt.month.astype('int8')


Category,DrillCompany,Rig-AcQuire,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses
0,BurGeoProekt,ZIF1200_01-BGP,2020,2,Direct,0.0,0.0,276.0
1,BurGeoProekt,ZIF1200_01-BGP,2020,3,Direct,47.0,420.0,278.0
2,BurGeoProekt,ZIF1200_01-BGP,2020,4,Direct,17.0,182.0,28.0
3,BurGeoProekt,ZIF1200_01-BGP,2021,1,Direct,9.0,7015.0,93.0
4,BurGeoProekt,ZIF1200_01-BGP,2021,2,Direct,84.0,397.0,191.0
...,...,...,...,...,...,...,...,...
2804,TechnoService-Eng,ZIF1200_16-TSE,2023,3,Direct,89.0,509.0,90.0
2805,TechnoService-Eng,ZIF1200_16-TSE,2023,4,Direct,76.0,468.0,106.0
2806,TechnoService-Eng,ZIF1200_16-TSE,2023,5,Direct,76.0,486.0,119.0
2807,TechnoService-Eng,ZIF1200_16-TSE,2023,6,Direct,77.0,405.0,133.0


In [241]:
import calendar

def hours_in_month(row):
    days = calendar.monthrange(row['year'], row['month'])[1]
    return days * 24

pivot_df['h_in_month'] = pivot_df.apply(hours_in_month, axis=1)

pivot_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2809 entries, 0 to 2808
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2809 non-null   object 
 1   Rig-AcQuire                2809 non-null   object 
 2   year                       2809 non-null   int16  
 3   month                      2809 non-null   int8   
 4   Tipe of circulation        2809 non-null   object 
 5   Planned_downtime           2809 non-null   float64
 6   Speed_losess               2809 non-null   float64
 7   Unplanned_downtime_losses  2809 non-null   float64
 8   h_in_month                 2809 non-null   int64  
dtypes: float64(3), int16(1), int64(1), int8(1), object(3)
memory usage: 162.0+ KB


In [242]:
df_meters['year'] = df_meters['ENDDATE'].dt.year.fillna(0).astype('int16')
df_meters['month'] = df_meters['ENDDATE'].dt.month.fillna(0).astype('int8')


df_meters

,HOLEID,DrillCompany,DrillRig,HolePurpose,HoleStatus,ENDDATE,DEPTH,year,month
0,MCI_N_227_1,TechnoService-Eng,ZIF1200_08-TSE,PIEZO,LIQUID,2024-11-16,421.0,2024,11
1,MCI_N_233_1,Mechkolonna,ZIF1200_19-MK49,PIEZO,ACCEPTED,2025-01-01,424.0,2025,1
2,MCI_N_236_1,BurGeoProekt,ZIF1200_09-BGP,PIEZO,ACCEPTED,2024-12-09,417.0,2024,12
3,MCI_N_245_1,KATCO,ZMO1500-5-KAT,PIEZO,CONSTRUCTION,NaT,417.0,0,0
4,MCI01_01_01_B_,KATCO,None,INJECT,ACCEPTED,2013-09-16,415.0,2013,9
...,...,...,...,...,...,...,...,...,...
17901,TSU77_09_06_C_,BurServisSnab,PRAKLA_01-BSS,INJECT,ACCEPTED,2024-07-31,344.0,2024,7
17902,TSU77_09_07_C_,Mechkolonna,ZIF1200_15-MK49,INJECT,ACCEPTED,2023-10-11,360.0,2023,10
17903,TSU77_09_08_C_,Mechkolonna,ZIF1200_21-MK49,INJECT,ACCEPTED,2023-10-09,350.0,2023,10
17904,TSU77_10_07_C_,BurGeoProekt,ZIF1200_03-BGP,PRODUCT,ACCEPTED,2024-07-08,359.0,2024,7


In [243]:
import numpy as np

df_meters = df_meters.dropna(subset=['year', 'month', 'DEPTH'])

pivot_meters = df_meters.pivot_table(
    index=['DrillRig', 'year', 'month'],  # группируем по году, месяцу и статусу
    columns='HoleStatus',
    values='DEPTH',
    aggfunc='sum',
    fill_value=0  # если нужно заполнить NaN нулями
).reset_index()

pivot_meters = pivot_meters.sort_values(
    by=['DrillRig', 'year', 'month'],
    ascending=[True, True, True]
).reset_index(drop=True)


# 1. Добавляем TOTAL_DRILLED
pivot_meters['TOTAL_DRILLED'] = (
    pivot_meters['ACCEPTED'] +
    pivot_meters['LIQUID'] +
    pivot_meters['NOT PROFITABLE']
)

# 2. Рассчитываем коэффициент (ACCEPTED + NOT PROFITABLE) / TOTAL_DRILLED
pivot_meters['ACCEPTED_NOTPROF_RATIO'] = (
    (pivot_meters['ACCEPTED'] + pivot_meters['NOT PROFITABLE']) /
    pivot_meters['TOTAL_DRILLED']
).replace([np.inf, -np.inf], np.nan).fillna(0)

# 3. Оставляем только нужные колонки
pivot_meters = pivot_meters[[
    'DrillRig', 'year', 'month', 'TOTAL_DRILLED', 'ACCEPTED_NOTPROF_RATIO'
]]


pivot_meters


HoleStatus,DrillRig,year,month,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO
0,DBKAZ40-1,2007,11,486.20,1.000000
1,DBKAZ40-1,2007,12,949.60,0.503370
2,DBKAZ40_1-KAT,2007,7,480.20,0.000000
3,DBKAZ40_1-KAT,2007,8,1935.60,0.754081
4,DBKAZ40_1-KAT,2007,9,1438.00,1.000000
...,...,...,...,...,...
5888,ZMO1500-6-KAT,2025,1,1677.00,1.000000
5889,ZMO1500-6-KAT,2025,2,1279.00,1.000000
5890,ZMO1500-6-KAT,2025,3,1686.25,1.000000
5891,ZMO1500-6-KAT,2025,4,1275.00,1.000000


In [244]:
pivot_df['Rig-AcQuire'] = pivot_df['Rig-AcQuire'].str.strip().str.upper()
pivot_meters['DrillRig'] = pivot_meters['DrillRig'].str.strip().str.upper()


total_merged_df = pd.merge(
    pivot_df,
    pivot_meters,
    left_on=['Rig-AcQuire', 'year', 'month'],
    right_on=['DrillRig', 'year', 'month'],
    how='left'  
)

total_merged_df.drop(columns='Rig-AcQuire', inplace=True)

total_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2809 entries, 0 to 2808
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   DrillCompany               2809 non-null   object 
 1   year                       2809 non-null   int16  
 2   month                      2809 non-null   int8   
 3   Tipe of circulation        2809 non-null   object 
 4   Planned_downtime           2809 non-null   float64
 5   Speed_losess               2809 non-null   float64
 6   Unplanned_downtime_losses  2809 non-null   float64
 7   h_in_month                 2809 non-null   int64  
 8   DrillRig                   2412 non-null   object 
 9   TOTAL_DRILLED              2412 non-null   float64
 10  ACCEPTED_NOTPROF_RATIO     2412 non-null   float64
dtypes: float64(5), int16(1), int64(1), int8(1), object(3)
memory usage: 205.9+ KB


In [245]:
total_merged_df['Planned_production_time'] = total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']

total_merged_df['Planned_factor'] = total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']

# 1. Planned Production Time (PPT) с ограничением снизу
total_merged_df['Planned_production_time'] = (
    total_merged_df['h_in_month'] - total_merged_df['Planned_downtime']
).clip(lower=0)

# 2. Planned Factor (в долях или в %)
total_merged_df['Planned_factor'] = (
    total_merged_df['Planned_production_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df.describe(include="all")


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor
count,2809,2809.000000,2809.000000,2809,2809.000000,2809.000000,2809.000000,2809.000000,2412,2412.000000,2412.000000,2809.000000,2809.000000
unique,5,NaN,NaN,2,NaN,NaN,NaN,NaN,66,NaN,NaN,NaN,NaN
top,BurGeoProekt,NaN,NaN,Direct,NaN,NaN,NaN,NaN,ZIF1200_29-BGP,NaN,NaN,NaN,NaN
freq,1057,NaN,NaN,2753,NaN,NaN,NaN,NaN,61,NaN,NaN,NaN,NaN
mean,NaN,2022.601994,6.271983,NaN,81.564970,468.616412,142.162157,729.663225,NaN,1204.038951,0.932183,648.098256,0.888242
std,NaN,1.451866,3.505805,NaN,31.601803,247.913885,179.713711,20.161504,NaN,527.493165,0.179872,35.834341,0.043195
min,NaN,2020.000000,1.000000,NaN,0.000000,0.000000,0.000000,672.000000,NaN,80.000000,0.000000,379.000000,0.509409
25%,NaN,2021.000000,3.000000,NaN,69.000000,426.000000,72.000000,720.000000,NaN,869.000000,1.000000,630.000000,0.866667
50%,NaN,2023.000000,6.000000,NaN,85.000000,496.000000,106.000000,744.000000,NaN,1090.500000,1.000000,648.000000,0.883929
75%,NaN,2024.000000,9.000000,NaN,97.000000,541.000000,156.000000,744.000000,NaN,1537.000000,1.000000,667.000000,0.905556


In [247]:
# Gross Operating Time (GOT)
total_merged_df['Gross_operating_time'] = total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']

# Availability (доступность)
total_merged_df['Availability'] = total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']

# Gross Operating Time (GOT) с ограничением снизу
total_merged_df['Gross_operating_time'] = (
    total_merged_df['Planned_production_time'] - total_merged_df['Unplanned_downtime_losses']
).clip(lower=0)

# Availability (доступность) с ограничением снизу
total_merged_df['Availability'] = (
    total_merged_df['Gross_operating_time'] / total_merged_df['h_in_month']
).clip(lower=0)


total_merged_df


,DrillCompany,year,month,Tipe of circulation,Planned_downtime,Speed_losess,Unplanned_downtime_losses,h_in_month,DrillRig,TOTAL_DRILLED,ACCEPTED_NOTPROF_RATIO,Planned_production_time,Planned_factor,Gross_operating_time,Availability
0,BurGeoProekt,2020,2,Direct,0.0,0.0,276.0,696,ZIF1200_01-BGP,80.0,0.000000,696.0,1.000000,420.0,0.603448
1,BurGeoProekt,2020,3,Direct,47.0,420.0,278.0,744,ZIF1200_01-BGP,704.0,1.000000,697.0,0.936828,419.0,0.563172
2,BurGeoProekt,2020,4,Direct,17.0,182.0,28.0,720,ZIF1200_01-BGP,697.0,1.000000,703.0,0.976389,675.0,0.937500
3,BurGeoProekt,2021,1,Direct,9.0,7015.0,93.0,744,NaN,NaN,NaN,735.0,0.987903,642.0,0.862903
4,BurGeoProekt,2021,2,Direct,84.0,397.0,191.0,672,ZIF1200_01-BGP,1001.0,1.000000,588.0,0.875000,397.0,0.590774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,TechnoService-Eng,2023,3,Direct,89.0,509.0,90.0,744,ZIF1200_16-TSE,598.0,1.000000,655.0,0.880376,565.0,0.759409
2805,TechnoService-Eng,2023,4,Direct,76.0,468.0,106.0,720,ZIF1200_16-TSE,1150.0,1.000000,644.0,0.894444,538.0,0.747222
2806,TechnoService-Eng,2023,5,Direct,76.0,486.0,119.0,744,ZIF1200_16-TSE,574.0,1.000000,668.0,0.897849,549.0,0.737903
2807,TechnoService-Eng,2023,6,Direct,77.0,405.0,133.0,720,ZIF1200_16-TSE,1063.0,1.000000,643.0,0.893056,510.0,0.708333
